# Preprocessing

This notebook contains code to read in data from the Muse and break it into labelled chunks.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.stats

In [2]:
%matplotlib notebook

## Importing data

In [5]:
# def loadData(filename='...')
# alldata = pd.read_csv("...")